# 一、语言与语法

1. 首先需要构建一个简单的便于形成迭代模型的语言定义
1. 而后需要构建语言转换规则，使得可以应用于成为优化调节
1. 不进行解题，而是只用于优化，即问题的化解

# 二、以排序为例

## 1. 输入
Input: x[5]

## 2. 处理
### 做或是求解 ，先考虑Do，优化的Do，而不考虑反解问题
Do or Slove: 

``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_ind_If=if(x_1<x_2)
  x_val_If=if(x[x_2]<x[x_1])
  x_If=x_ind_If & x_val_If
  swap(x_If)(x,x_i,x_k)
``` 

## 3. 输出
Output: x[5]

# 3.1 模型重构

## 1 语言实现v0.1

``` go
for(x_1 :0..1..x.len):
 for(x_2 :0..1..x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  x_ind_If=if(x_1<x_2)
  x_val_If=if(x[x_2]<x[x_1])
  x_If=x_ind_If & x_val_If
  swap(x_If)(x,x_i,x_k)
``` 

####  问题列表
1. 所有的数据是一个集合，如x_1,x_2,x
1. 操作是对集合的遍历导致的操作，或者对集合的元素进行操作
1. 函数是需要有输入，有输出
1. 函数出入参的传递是生存周期问题
1. 操作是有目的性的，目的性决定了优化性
1. 变量的产生需要定义，需要销毁
1. 表述，及意义在 代码的表达形式是次要的，内涵是一致的


## 2 语言实现v0.2


``` python
for x_1 in range(x.len):
 for x_2 in range(x.len):
  x_1_val=x[x_1]
  x_2_val=x[x_2]
  if(x_1<x_2 and x[x_2]<x[x_1]):
    x_t=x[x_i]
    x[x_i]=x[x_k]
    x[x_k]=x_t
```


#### 问题解决列表
1. 层次结构转为==>函数调用结构
1. 消除了变量的生存期，生存期只在函数内
1. 理论上也支持递归
1. 只有层级化解为调用关系，函数内存在执行顺序(依赖顺序，而不是严格顺序),依赖顺序是由于变量变化导致的顺序依赖，严格顺序是次序造成的顺序
1. 输入函数参数比较杂乱，输入变量也是输出变量不够好


``` python
# action:   FOR,IF
# function:  Main,F1,F2,F3

def Main(x/x):
    FOR(x_1:0..1..x.len,F1(x,x_1))
def F1(x,x_1/x):
    FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
def F2(x,x_1,x_2/x):
    x_1_val=GV(x,x_1)
    x_2_val=GV(x,x_2)
    x_If_1=x_1<x_2
    x_If_2=x_2_val<x_1_val
    x_If=x_If_1 and x_If_2
    IF(x_If,F3(x,x_1,x_2))
def F3(x,x_1,x_2/x):
    x_t=GV(x,x_1)
    SV(x,GV(x,x_2))
    SV(x,x_t)

```

#### 代码即需求，代码即结构

``` go
Ma[.] -> FOR(x_l,F1[...])
F1[.] -> FOR(x_2,F2[...])
F2[.] -> 1. 2. 3. 4. 5. IF(.,F3[...])
F3[.] -> 1. 2. 3.

```

#### 优化特征识别
``` go
FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
F2(..):
    ...//无x_2
    x_If_1=x_1<x_2
    ... && x_If

==>

FOR(x_2:x_1..1..x.len,F2(x,x_1,x_2))
F2(..):
    ...//无x_2
    //x_If_1=x_1<x_2
    ... && true

```

#### 灵感获得路径

#### 特征项获得是否满足条件，及改造方式的定义
```go
//运行及执行操作
//输入什么，代码片段，还是优化行
//特征提取方式近似于解析过程,正则？脚本？还是特殊执行体
def check1(F1[1:FOR..],x_2,F2[3:x_If_1..]):
  x_2 ==> 0..1..x.len
  if_pos //IF改变的位置
  x_2_toTrue //改变的代码位置
  while(getop(F1)):
    switch(op):
        FOR:
          //进入函数
          //没有其他x_2依赖项
        FUN:
          //没有其他x_2依赖项
        IF:
          if(x_1<x_2) //可作为条件，正好是F2的函数
          //
          //没有其他x_2依赖项
        Op:
          //其他执行体
          //没有其他x_2依赖项
          //使用x_1<x_2的位置记录下来
   return false or true
```

#### 以上结果偏复杂，难以逻辑组合、集成、改型
```python
def check1(F1[1:FOR..],x_2,F2[3:x_If_1..]):
    #产生遍历表
     '''产生x_2的判断逻辑的周期
     F1:
       FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
        -->x_1_val=GV(x,x_1),x_2_val=GV(x,x_2)
      | -->x_If_1=x_1<x_2 *
      | -->x_If_2=GV(x,x_2)<GV(x,x_1)
      | -->x_If=x_If_1 and x_If_2,IF(x_If,F3(x,x_1,x_2))
      ...产生依赖关系图
     
     '''
```
#### 产生结论
1. 是否有找到符合条件的项(有>0的某个x_2范围没有使用),并且所有执行导致输出都与x_2的范围有关，不存在其他导致输出的操作
2. FOR发生的位置，及IF发生的位置s
3. IF结果需要替换的位置s
4. 有一个不完备的地方，不出现x_2的部分可能引发输出，提取后无法保障输出的一致性，如果确保x_2改型后不导致引发问题？

#### 尝试优化为输入输出分离? 会变复杂化

``` python
# action:   FOR,IF
# function:  Main,F1,F2,F3

def Main(x/x):
    FOR(x_1:0..1..x.len,F1(x,x_1))
def F1(x,x_1/x):
    FOR(x_2:0..1..x.len,F2(x,x_1,x_2))
def F2(x,x_1,x_2/x):
    x_1_val=GV(x,x_1)
    x_2_val=GV(x,x_2)
    x_If_1=x_1<x_2
    x_If_2=x_2_val<x_1_val
    x_If=x_If_1 
    x_If=x_If and x_If_2
    IF(x_If,F3(x,x_1,x_2))
def F3(x,x_1,x_2/x):
    x_t=GV(x,x_1)
    SV(x,GV(x,x_2))
    SV(x,x_t)

```


#### 变形表，包括生命周期和输入输出依赖
* 01. FOR(x_1:0..1..x.len,F1(x,x_1))  --> x(01)
* 02. FOR(x_2:0..1..x.len,F2(x,x_1,x_2)) --> x(02)
* 03. x_1_val=GV(x,x_1) --> x_1_val(03)
* 04. x_2_val=GV(x,x_2) --> x_2_val(04)
* 05. x_If_1=x_1<x_2 --> x_If_1(05)
* 06. x_If_2=x_2_val<x_1_val --> x_If_2(06)
* 07. x_If=x_If_1 --> x_If(07)
* 08. x_If=x_If and x_If_2 --> x_If(08)
* 09. IF(x_If,F3(x,x_1,x_2)) --> x(08)
* 10. x_t=GV(x,x_1) --> x_t(09)
* 11. SV(x,GV(x,x_2)) --> x(10)
* 12. SV(x,x_t) --> x(11)

In [143]:
# 定义数据

## 目前测试不是非常适合用有向图解决，因为缺乏函数或变量的生存时间性质等

showJson1='''
{ "class": "GraphLinksModel",
  "nodeKeyProperty": "id",
  "nodeDataArray": [ 
{"id":0, "loc":"211.35000000000005 79.05000000000004", "text":"Main"},
{"id":1, "loc":"365.0738095238096 75.3738095238095", "text":"FOR1"},
{"text":"F1", "loc":"541.9738095238097 69.07380952380952", "id":-3}
 ],
  "linkDataArray": [ 
{"from":0, "to":1, "text":"x", "curviness":20, "points":[258.26989528570044,80.07309427671363,294.2507454351132,44.67136260315783,339.98756173697046,42.529161561311795,376.19120901808844,75.98313063667635]},
{"from":1, "to":0, "points":[376.8303907503867,113.9096720037289,331.33725071996804,159.73632585135312,282.4129142047914,151.53466868394838,255.31333907370026,117.81825071925174], "text":"x"},
{"from":1, "to":-3, "text":"transition", "points":[422.8780689384324,79.74374974881732,464.6701392050182,56.35193213230113,505.7955368643314,56.97485624963076,543.1160319001649,77.99426554283292]},
{"from":-3, "to":1, "points":[544.9316531167991,101.8194441034216,500.88188632621035,136.25143010302997,461.5222364280437,127.38814349533808,423.86327713149785,108.70462369079485]}
 ]}
'''

# 展示网站
from IPython.display import HTML
f = open('Show.html','rb')
data = f.read()                   # 读取文件内容
f.close()
HTML(data.replace("showJson1",showJson1))

# 三、初步实现与建模

## 1 模型转换
#### 定义过程型
|P4Short|P     |复杂度  |  描述|
|----|----     |---  |----|
|FR  |ForRange |x   |   遍历输入量x产生输出集合0..1..x,作为一个块看待,可输出ind，以及标识结束，作为后续依赖的标志|
|IS  |IfSmall  |1   |   判断是否左x1小于右侧x2|
|GS  |GetSize  |1   | 获得x集合的长度|
|GV  |GetValue |1   | 获得x结合的indx位置的值|
|SV  |SetValue |1   | 设置x结合的indx位置的值|
|LA  |LogicAnd |1   | 逻辑与运算|
|SW  |swap     |1  | 交换x的ind1和ind2位置
|IM  |InsertMap|log(x)    |插入Map数据结构|
|GM  |GetMap|1    |获得Map结构中的一项|

|说明|
|:---|
| 因此依赖关系针对FR分为两种，一种为获得其中一项作为FR的内部依赖，一种为FR的结束依赖|
| FR是函数的抽象，普通函数为FR为1的函数|
|函数本身也决定了一种独特的依赖关系，内部依赖和外部依赖，是否可以将两种依赖统一呢|
|目前两种依赖关系等效于：即非树形，也非完全的AST语法树|

#### 过程输入输出表（图?CNN?）

##### P2，P3输入输出上没有体现出依赖关系，但是实质是存在依赖性的，即P3必须依赖于P2，是否依赖关系要重新建立一张表呢？

|N  |P    |P_pre  |    x|  x_len|   x_1|   x_2| x_1_val |x_2_val | x_ind_If | x_val_If| x_If|
|---|---  |---    |---  |---   |---   |---   |---    |---     |---  | ----|  ----|
|  1| GS  |0     |   I  |    O |
|  2| FR  |1     |&nbsp; |    I |    O|
|  3| FR  |2     |&nbsp; |    I |&nbsp;|    O|
|  4| GV  |3     | I    |&nbsp; | I   |&nbsp;| O|   
|  5| GV  |3     | I    |&nbsp; |&nbsp;|I    | &nbsp; |       O|
|  6| IS  |3     | &nbsp;| &nbsp;|   I |    I| &nbsp; |  &nbsp;  |    O|
|  7| IS  |3     | &nbsp;| &nbsp;|&nbsp;|&nbsp;| I     |  I |   &nbsp;|  O |
|  8| LA  |4,5,6,7 | &nbsp;| &nbsp;|&nbsp;|&nbsp;| &nbsp;  |&nbsp; |   I|  I | O|
|  9| SW  |8     | O    |&nbsp; |I    | I   | &nbsp; | &nbsp;   |&nbsp;|  &nbsp;|I|

#### 依赖关系表
|N  |P1    |P2    |P3    |P4    |P5    |P6    |P7    |P8
|---|---  |---    |---  |---   |---   |---   |---    |--- |
| P1|
| P2|Y|
| P3|&nbsp;|Y|
| P4|&nbsp;|Y|
| P5|&nbsp;|&nbsp;|Y|
| P6|&nbsp;|Y|Y|
| P7|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P8|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|Y|
| P9|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|&nbsp;|Y|


## 2 复杂度计算

``` go
P1 = 1
P2 = P1 * (x_len) 
P3 = P2 * (x_len)
P4 = P3 * (1)
P5 = P3 * (1)
P6 = P2 + P3
P7 = P4 + P5
P8 = P6 + P7
P9 = P8

==>

P9 = P2 + P3 + P4 + P5
  = P1 * (x_len) + P1 * (x_len) * (x_len) + P1 * (x_len) * (x_len) + P1 * (x_len) * (x_len)
  = P1 * (x_len) * (x_len)
  ~= N * (x_len) * (x_len)
```